R environment

In [ ]:
suppressPackageStartupMessages({
    library(Seurat)
    library(dplyr)
    library(future)
    plan("multiprocess", workers = 12)
    options(future.globals.maxSize = 1000 * 1024^8)
    options(stringsAsFactors = FALSE)
    set.seed(123)
})

Warning message:
“Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'.”


In [ ]:
reference <- readRDS('data/core_GBmap.rds')

### Load query

In [ ]:
use_condaenv("scarches")
sc <- import("scanpy")

In [ ]:
gbm <- readRDS('data/query_GB_datasets.rds')

In [ ]:
gbm

An object of class Seurat 
91847 features across 797113 samples within 3 assays 
Active assay: RNA (91773 features, 2000 variable features)
 2 other assays present: prediction.score.high_hierarchy, prediction.score.low_hierarchy
 2 dimensional reductions calculated: pca, ref.pca

In [ ]:
gbm <- NormalizeData(gbm)%>%FindVariableFeatures()%>%ScaleData()%>%RunPCA()
gbm

Centering and scaling data matrix

PC_ 1 
Positive:  CTSD, S100A4, CCL4L2, IL1B, CCL3L1, HMOX1, CTSL, CXCL8, S100A6, PLIN2 
	   LYZ, NDUFB2, S100A9, HLA-DRB5, FCGBP, CFD, GPNMB, PLTP, IFITM3, TGFBI 
	   MT2A, S100A8, TAGLN2, CH25H, G0S2, ZNF331, HAMP, TIMP1, LY6E, APOC2 
Negative:  LSAMP, CADM2, PPP2R2B, DLG2, LRP1B, NCAM2, IL1RAPL1, AKAP6, PTPRD, CTNNA2 
	   DNM3, ANKS1B, FRMD5, KAZN, ERBB4, NRXN1, SOX5, DMD, LRRC4C, PTPRK 
	   DLGAP1, CSMD1, NRXN3, SOX6, GRIK2, SNTG1, LHFPL3, DSCAML1, GALNT13, ASTN2 
PC_ 2 
Positive:  SOX5, SNTG1, SOX6, NRXN1, SLC35F1, MMP16, KCND2, DLGAP1, LHFPL3, GLIS3 
	   NKAIN3, THSD7A, BRINP3, KALRN, PLCB1, GRIK2, EGFR, ROBO2, EGFEM1P, SLC4A4 
	   FGF14, DCLK1, GRIA1, RGS6, TNC, SLC1A2, CACNB2, FAT3, OPCML, NXPH1 
Negative:  SPOCK3, MOG, ST18, CNDP1, EDIL3, ENPP2, TF, AK5, PCSK6, MOBP 
	   TMEM144, NKAIN2, SLC24A2, PLEKHH1, PEX5L, CNTNAP4, MAP7, C10orf90, GRM3, SLC5A11 
	   DOCK5, LRP2, SH3GL3, KLHL32, MBP, PIEZO2, MAG, CTNNA3, PLP1, FGFR2 
PC_ 3 
Positive:  FR

An object of class Seurat 
91773 features across 797113 samples within 1 assay 
Active assay: RNA (91773 features, 2000 variable features)
 1 dimensional reduction calculated: pca

### Reference mapping

In [ ]:
reference <- FindVariableFeatures(reference, selection.method = "vst", nfeatures = 2000)
reference <- ScaleData(reference, features = VariableFeatures(object = reference), verbose = FALSE)
reference <- RunPCA(reference, features = VariableFeatures(object = reference), verbose = FALSE)

In [ ]:
anchors <- FindTransferAnchors(
  reference = reference,
  query = gbm,
  features = rownames(reference[["RNA"]]),
  normalization.method = "LogNormalize",
  reference.reduction = "pca",
  dims = 1:50
)

Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 265885 anchors

Filtering anchors

	Retained 38658 anchors



`Map Query` functions run separately

In [ ]:
gbm <- TransferData(
  anchorset = anchors, 
  reference = reference,
  query = gbm,
  refdata = list(
      high_hierarchy = 'annotation_level_3',
      low_hierarchy = 'annotation_level_4'
  ),
)

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from predictionscorehigh_hierarchy_ to predictionscorehighhierarchy_”
Predicting cell labels

Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from predictionscorelow_hierarchy_ to predictionscorelowhierarchy_”


In [ ]:
gbm <- IntegrateEmbeddings(
  anchorset = anchors,
  reference = reference,
  query = gbm, 
  new.reduction.name = "ref.pca"
)


Integrating dataset 2 with reference dataset

Finding integration vectors

Integrating data

Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_lapply-1’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "ignore".”


In [ ]:
saveRDS(gbm, 'data/mapped_query_GB_datasets.rds')